# I310D Homework 2
## Deadline: 11:59 pm, October 17, 2022

### Exploring the dataset

Compare the scores to the labels assigned by manual reviewers. 

Maybe do some data parsing to find out things like the most common words used in comments where toxicity or abuse was present. Where did the model make mistakes?
Test the API for potential biases.


In [14]:
import pandas as pd
import time

df = pd.read_csv('labeled_and_scored_comments.csv')
df

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
41333,9480bff99f91a69e,Lesbian Teenager \nHow about the fact she was ...,0.797672,1,0,0,0,0,0
41334,9480115b93762c35,"Crack, also called cocaine is very good for yo...",0.240977,0,0,0,0,0,0
41335,947f206c14a85d9d,REDIRECT Talk:The Test (The O.C.),0.012671,0,0,0,0,0,0
41336,947e3f5f2d613605,A question concerning your edit to my user-pag...,0.227474,0,0,0,0,0,0


In [59]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = 'XXXXXXXXXXXXXXXXXXX' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

### Designing and performing tests

Determine a threshold for which scores are considered toxic or abusive.

In [60]:
threshold = 0.6;

Here, I am testing out a few things I am curious about

In [66]:
comment_list = ['hate', 'you are so ugly', 'please die', 'you are ugly', 'i hope you die']
score_list = []

We will insert a time.sleep() statement between consecutive API calls (pauses execution for one second)

In [67]:
for comment in comment_list:
    score = get_toxicity_score(comment)
    score_list.append(score)
    time.sleep(1)
    
sum(score_list)

3.69107685

Surprisingly enough, adding "so" to "you are ugly" increased the toxicity score by about 0.05 & that made it higher than the "please die" and "i hope you die" comments. However, I was not surprised that "please die" had a lower score than "i hope you die", and I'm assuming it's because of the word "please".

In [68]:
new_df = pd.DataFrame({'comment': comment_list, 'score': score_list})
new_df.head()

,comment,score
0,hate,0.271282
1,you are so ugly,0.883106
2,please die,0.830203
3,you are ugly,0.839334
4,i hope you die,0.867151


Hypothesis about Perspective’s performance (based on my understanding of how the API is trained and used):
> My hypothesis is that Perspective will make mistakes on passive aggressive toxic comments and rate them with a lower toxicity score even if they are just as harmful as the straightforward, agressive comments.       
     
Develop small test set, document model scores, and assess whether or not the hypothesis was correct based on the sample.

Here, I am choosing to evaluate the'threat' and 'insult' coments because I am curious to see how many are fale positives and false negatives.

In [78]:
from sklearn.metrics import confusion_matrix

df['prediction'] = (df['score'] > threshold).astype(int)
df['prediction'].value_counts()

df[((df['threat']==1)|(df['insult']==1))&(df['prediction']==0)]

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
3107,163374fb3002bd1c,==Scruffy== \n\n That was just rude. I was tr...,0.534903,0,0,0,0,1,0,0
3109,d656aecfa2f205bb,"No, and you're a lamer",0.537578,1,0,0,0,1,0,0
3704,668122442c49d0d4,""" \n :hmmmmm.... you speak in riddles, my frie...",0.553224,1,0,0,0,1,0,0
7094,e6630d659a17c104,Is the Ellie Chronicals going to continue and ...,0.476537,0,0,0,0,1,0,0
8396,d1822fe6fde979ab,Oh go away Artkos. Go and make a total pest o...,0.587656,0,0,0,0,1,0,0
8428,d11b543bb45c185a,[[==:Amortias is M(o)ther (fu)k(er)==,0.466100,1,0,1,0,1,0,0
8441,d0dbf37a37f078f3,Dave Wain says so the moderator thinks that an...,0.563029,1,0,0,0,1,0,0
9673,bcac23e93c23e725,I'm not trying to spread propaganda. I'm merel...,0.563575,0,0,1,0,1,0,0
11489,9e63db9f1dbbd99a,Hey \n\nAnd you're telling me that without a g...,0.310894,1,0,0,0,1,0,0
11495,9e4d715314efdb7b,LISTEN YOU LOT A RE BNUNJCH OF LITTEL HITLERS ...,0.536867,1,0,0,0,1,0,0


As a note to myself:

TP (top left) FP (top right)
                     
                     
FN (bottom left) TN (bottom right)

In [73]:
confusion_matrix(df['threat'], df['prediction'])

array([[36129,  5084],
       [    9,   116]], dtype=int64)

In [74]:
confusion_matrix(df['insult'], df['prediction'])

array([[36085,  3194],
       [   53,  2006]], dtype=int64)

How does a low sample size impact your results, and the conclusions we can draw from them?
> A low sample size can narrow down our results, and depending on how small the sample size is, that could be a good or bad thing. A sample size that is too small can lead to false assumptions. In this case, we are just narrowing down our sample size to labels that we need or are interested in. 

### Analyzing the data

How did false positive rates and false negative rates compare against different types of content? 
>Threats had many more false positives than insults, while insults had more false negatives than threats

Were there any cases where you disagreed with the labels? 
>I felt that some cases that were labeled as "threats" were not really threats.

Were there any trends that you noticed in the model scores? What about any original queries that you submitted?
>Most toxic or severely toxic comments had 'insult' == 1, which isn't too surprising since most toxic comments are in the form of insulting another person

### My Results

What (if anything) surprised you about your findings, and/or what theories you have about why any biases might exist, if you find they exist. 
>

What biases do you think might exist in the model based on intuitions or public documentation about how the model was created?
>

What were your results?
>

What theories do you have about why your results are what they are?
>